# 概述
改进数解法    
南北向干道，5个交叉口[0,1,2,3,4]，全部交叉口单口放行    
基准交叉口为交叉口0    
间距：500, 380, 420, 140m    
速度：10m/s    
假设各相位的绿灯时间等于相位时间, 即各相位的绿信比之和为1    
公共信号周期范围：40-120s     
各交叉口绿信比设计：

|交叉口|南进口|北进口|东进口|西进口|
|---|---|---|---|---|
|0|0.34|0.28|0.22|0.16|
|1|0.3|0.28|0.2|0.22|
|2|0.32|0.32|0.2|0.16|
|3|0.3|0.36|0.14|0.2|
|4|0.36|0.34|0.18|0.12|

In [315]:
import pandas as pd
import numpy as np

# parameters

spacing = [500, 380, 420, 140]    # unit: m

# ratio sequence: [S, N, E, W]
ratio_matrix = [[0.34, 0.28, 0.22, 0.16], 
                [0.3, 0.28, 0.2, 0.22],
                [0.32, 0.32, 0.2, 0.16], 
                [0.3, 0.36, 0.14, 0.2],
                [0.36, 0.34, 0.18, 0.12]]

# cycle range, unit: s
c_min = 90
c_max = 110

step = 2
v = 10    # unit: m/s

In [316]:
# valid phase sequence: 
# 0: SNEW
# 1: NSEW
# 2: SENW
# 3: SWNE
# 4: SNWE
# 5: NSWE
'''
seq_combine =  [[0, 1, 2, 3], 
                [1, 0, 2, 3], 
                [0, 2, 1, 3], 
                [0, 3, 1, 2]]
'''

'\nseq_combine =  [[0, 1, 2, 3], \n                [1, 0, 2, 3], \n                [0, 2, 1, 3], \n                [0, 3, 1, 2]]\n'

In [317]:
def getInterval(c):
    return 0.5 * v * c

def getDeltaT(idx, seq_idx, c):
    delta_t = 0.5*ratio_matrix[idx][0]*c + 0.5*ratio_matrix[idx][1]*c
    t_i = (1 - np.sum(ratio_matrix[idx])) / len(ratio_matrix[idx])

    if seq_idx == 0:
        delta_t += t_i
    elif seq_idx == 1:
        delta_t = -1 * delta_t 
        delta_t -= t_i
    elif seq_idx == 2:
        delta_t += ratio_matrix[idx][2]*c + 2*t_i
    elif seq_idx == 3:
        delta_t += ratio_matrix[idx][3]*c + 2*t_i

    return delta_t

def getIdealDis(a_idx, b_idx, a_seq_idx, b_seq_idx, c, n=0):
    #delta_t_a = 0.5*ratio_matrix[a_idx][0]*c + 0.5*ratio_matrix[a_idx][1]*c + t_i
    delta_t_a = getDeltaT(a_idx, a_seq_idx, c)

    '''
    delta_t_b = 0.5*ratio_matrix[b_idx][0]*c + 0.5*ratio_matrix[b_idx][1]*c
    if b_seq_idx == 0:
        delta_t_b += t_i
    elif b_seq_idx == 1:
        delta_t_b = -1 * delta_t_b - t_i
    elif b_seq_idx == 2:
        delta_t_b += ratio_matrix[b_idx][2]*c + 2*t_i
    elif b_seq_idx == 3:
        delta_t_b += ratio_matrix[b_idx][3]*c + 2*t_i
    '''

    delta_t_b = getDeltaT(b_idx, b_seq_idx, c)

    res = 0.5*v*delta_t_a - 0.5*v*delta_t_b

    while res < 0:
        res += getInterval(c)

    return res

# 计算理想距离

In [318]:
idea_dis_cols = ['cycle']
for i in range(1, len(ratio_matrix)):
    for j in range(len(ratio_matrix[i])):
        name = str(i) + '-' + str(j)
        idea_dis_cols.append(name)
idea_dis_cols.append('interval')
print(idea_dis_cols)

idea_dis_matrix = []
idea_dis_table = []

for t in range(c_min, c_max+step, step):
    matrix_row = []
    table_row = [t]

    for i in range(1,5):
        for j in range(4):
            ideal_dis = getIdealDis(0, i, 1, j, t)
            matrix_row.append(ideal_dis)

    idea_dis_matrix.append(matrix_row)
    table_row += matrix_row
    table_row.append(getInterval(t))
    idea_dis_table.append(table_row)

idea_dis_df = pd.DataFrame(idea_dis_table, columns=idea_dis_cols)
idea_dis_df

['cycle', '1-0', '1-1', '1-2', '1-3', '2-0', '2-1', '2-2', '2-3', '3-0', '3-1', '3-2', '3-3', '4-0', '4-1', '4-2', '4-3', 'interval']


,cycle,1-0,1-1,1-2,1-3,2-0,2-1,2-2,2-3,3-0,3-1,3-2,3-3,4-0,4-1,4-2,4-3,interval
0,90,180.0,441.0,90.0,81.0,166.5,4.5,76.5,94.5,162.0,9.0,99.0,72.0,153.0,18.0,72.0,99.0,450.0
1,92,184.0,450.8,92.0,82.8,170.2,4.6,78.2,96.6,165.6,9.2,101.2,73.6,156.4,18.4,73.6,101.2,460.0
2,94,188.0,460.6,94.0,84.6,173.9,4.7,79.9,98.7,169.2,9.4,103.4,75.2,159.8,18.8,75.2,103.4,470.0
3,96,192.0,470.4,96.0,86.4,177.6,4.8,81.6,100.8,172.8,9.6,105.6,76.8,163.2,19.2,76.8,105.6,480.0
4,98,196.0,480.2,98.0,88.2,181.3,4.9,83.3,102.9,176.4,9.8,107.8,78.4,166.6,19.6,78.4,107.8,490.0
5,100,200.0,490.0,100.0,90.0,185.0,5.0,85.0,105.0,180.0,10.0,110.0,80.0,170.0,20.0,80.0,110.0,500.0
6,102,204.0,499.8,102.0,91.8,188.7,5.1,86.7,107.1,183.6,10.2,112.2,81.6,173.4,20.4,81.6,112.2,510.0
7,104,208.0,509.6,104.0,93.6,192.4,5.2,88.4,109.2,187.2,10.4,114.4,83.2,176.8,20.8,83.2,114.4,520.0
8,106,212.0,519.4,106.0,95.4,196.1,5.3,90.1,111.3,190.8,10.6,116.6,84.8,180.2,21.2,84.8,116.6,530.0
9,108,216.0,529.2,108.0,97.2,199.8,5.4,91.8,113.4,194.4,10.8,118.8,86.4,183.6,21.6,86.4,118.8,540.0


# 计算偏移距离

In [319]:
bias_cols = ['cycle']
bias_cols += [i for i in range(1, len(ratio_matrix))]
bias_cols.append('interval')

bias_matrix = []
bias_table = []

dis = [0]
for i in range(1, len(spacing)+1):
    dis.append(dis[i-1]+spacing[i-1])
print(dis)

for t in range(c_min, c_max+step, step):
    matrix_row = []
    table_row = [t]
    interval = getInterval(t)

    for i in range(1, len(spacing)+1):
        bias = dis[i] % interval

        matrix_row.append(bias)

    table_row += matrix_row
    table_row.append(interval)
    bias_matrix.append(matrix_row)
    bias_table.append(table_row)
    
print(bias_matrix)
bias_df = pd.DataFrame(bias_table, columns=bias_cols)
bias_df

[0, 500, 880, 1300, 1440]
[[50.0, 430.0, 400.0, 90.0], [40.0, 420.0, 380.0, 60.0], [30.0, 410.0, 360.0, 30.0], [20.0, 400.0, 340.0, 0.0], [10.0, 390.0, 320.0, 460.0], [0.0, 380.0, 300.0, 440.0], [500.0, 370.0, 280.0, 420.0], [500.0, 360.0, 260.0, 400.0], [500.0, 350.0, 240.0, 380.0], [500.0, 340.0, 220.0, 360.0], [500.0, 330.0, 200.0, 340.0]]


,cycle,1,2,3,4,interval
0,90,50.0,430.0,400.0,90.0,450.0
1,92,40.0,420.0,380.0,60.0,460.0
2,94,30.0,410.0,360.0,30.0,470.0
3,96,20.0,400.0,340.0,0.0,480.0
4,98,10.0,390.0,320.0,460.0,490.0
5,100,0.0,380.0,300.0,440.0,500.0
6,102,500.0,370.0,280.0,420.0,510.0
7,104,500.0,360.0,260.0,400.0,520.0
8,106,500.0,350.0,240.0,380.0,530.0
9,108,500.0,340.0,220.0,360.0,540.0


# 计算偏移绿信比

In [320]:
offset_ratio_cols = ['cycle']
for i in range(1, len(ratio_matrix)):
    for j in range(len(ratio_matrix[i])):
        name = str(i) + '-' + str(j)
        offset_ratio_cols.append(name)

    offset_ratio_cols.append('optimal_seq')

optimal_table_cols = ['cycle']
optimal_table_cols += [str(i) for i in range(1, len(spacing)+1)]
optimal_table_cols.append('optimal phase sequence')
optimal_table_cols.append('max offset ratio sum')

offset_ratio_matrix = []
offset_ratio_table = []
optimal_ratio_matrix = []
optimal_table = []

for t in range(len(idea_dis_matrix)):
    matrix_row = []
    table_row = [idea_dis_table[t][0]]
    optimal_matrix_row = []
    optimal_table_row = [idea_dis_table[t][0]]
    optimal_seq = []
    interval = idea_dis_table[t][-1]

    for i in range(1, len(spacing)+1):
        min_offset_ratio = float('inf')
        optimal_phase_seq_idx = -1

        for j in range(4):

            diff = bias_matrix[t][i-1] - idea_dis_matrix[t][(i-1)*4+j]
            #print("{} - {} = {}".format(bias_matrix[t][i-1], idea_dis_matrix[t][(i-1)*4+j], diff))
            while diff < 0 and abs(diff+interval) < abs(diff):
                diff += interval

            print("diff: {}".format(diff))
            offset_ratio = 0.5 * diff / interval

            matrix_row.append(offset_ratio)

            if np.abs(min_offset_ratio) > np.abs(offset_ratio):
                min_offset_ratio = offset_ratio
                optimal_phase_seq_idx = j

        matrix_row.append(optimal_phase_seq_idx)
        optimal_seq.append(optimal_phase_seq_idx)
        optimal_matrix_row.append(min_offset_ratio)

    table_row += matrix_row
    optimal_table_row += optimal_matrix_row
    optimal_table_row.append(optimal_seq)

    # max offset ratio sum
    sum = max(optimal_matrix_row) - min(optimal_matrix_row)
    optimal_table_row.append(sum)

    offset_ratio_table.append(table_row)
    offset_ratio_matrix.append(matrix_row)
    optimal_ratio_matrix.append(optimal_matrix_row)
    optimal_table.append(optimal_table_row)

print(offset_ratio_matrix)
offset_ratio_df = pd.DataFrame(offset_ratio_table, columns=offset_ratio_cols)
print(offset_ratio_df)
print(optimal_ratio_matrix)
optimal_df = pd.DataFrame(optimal_table, columns=optimal_table_cols)
offset_ratio_df.to_excel('optimal-ratio.xlsx')
optimal_data = optimal_df[optimal_df['max offset ratio sum'] == optimal_df['max offset ratio sum'].min()].iloc[0]
optimal_data

diff: -130.0
diff: 59.0
diff: -40.0
diff: -31.0
diff: 263.5
diff: 425.5
diff: 353.5
diff: 335.5
diff: 238.0
diff: 391.0
diff: 301.0
diff: 328.0
diff: -63.0
diff: 72.0
diff: 18.0
diff: -9.0
diff: -144.0
diff: 49.200000000000045
diff: -52.0
diff: -42.799999999999955
diff: 249.80000000000007
diff: 415.4
diff: 341.80000000000007
diff: 323.40000000000003
diff: 214.40000000000003
diff: 370.8
diff: 278.80000000000007
diff: 306.40000000000003
diff: -96.39999999999998
diff: 41.60000000000002
diff: -13.599999999999966
diff: -41.19999999999993
diff: -158.0
diff: 39.39999999999998
diff: -64.0
diff: -54.60000000000002
diff: 236.10000000000002
diff: 405.29999999999995
diff: 330.1
diff: 311.3
diff: 190.79999999999995
diff: 350.6
diff: 256.6
diff: 284.79999999999995
diff: -129.8
diff: 11.199999999999989
diff: -45.200000000000045
diff: -73.39999999999998
diff: -172.0
diff: 29.600000000000023
diff: -75.99999999999994
diff: -66.39999999999998
diff: 222.39999999999998
diff: 395.20000000000005
diff: 318.40

cycle                              110
1                            -0.035455
2                                0.115
3                             0.001818
4                             0.139091
optimal phase sequence    [1, 0, 0, 0]
max offset ratio sum          0.174545
Name: 10, dtype: object

# 计算相位差

In [321]:
phase_seq = optimal_data['optimal phase sequence']
phase_diff_list = []
ratio_s = []
ideal_pos_list = [0]
base_phase = 0
interval = getInterval(optimal_data['cycle'])
#print('interval: {}'.format(interval))

for i in ratio_matrix:
    ratio_s.append(i[0])

for i in range(1, len(ratio_matrix)):
    ideal_dis = getIdealDis(0, i, base_phase, phase_seq[i-1], optimal_data['cycle'])
    ideal_pos = ideal_dis

    # ideal pos
    dis_diff = dis[i] - ideal_dis
    while np.abs(dis_diff) > np.abs(dis_diff - interval):
        dis_diff -= interval
        ideal_pos += interval

    ideal_pos_list.append(ideal_pos)

for i in range(len(ratio_matrix)):
    phase_diff = ideal_pos_list[i]/v - 0.5*optimal_data['cycle']*ratio_s[i]
    while phase_diff > optimal_data['cycle']:
        phase_diff -= optimal_data['cycle']
    while phase_diff < 0:
        phase_diff += optimal_data['cycle']

    phase_diff_list.append(phase_diff)

print(ideal_pos_list)
print(ratio_s)
print(phase_diff_list)

[0, 330.00000000000006, 1094.5, 1089.0, 1628.0]
[0.34, 0.3, 0.32, 0.3, 0.36]
[91.3, 16.500000000000007, 91.85, 92.4, 33.0]


# 封装
针对基准交叉口的单个特定相序的情况进行封装

In [322]:
class LineCoordinationBase1Phase:
    def __init__(self, spacing, ratio_matrix, c_min, c_max, step, v, base_phase_seq):
        '''
        spacing: spacing of intersections in one line
        ratio_matrix: ratios of each intersections, sequence: SNEW
        c_min: minimum cycle, unit: s
        c_max: maximum cycle, unit: s
        step: cycle range will grow by step
        v: green wave speed, unit: m/s
        base_phase_seq: phase sequence of base intersection (intersection 0)

        Valid phase sequence: 
        0: SNEW
        1: NSEW
        2: SENW
        3: SWNE
        '''
        

        self.spacing = spacing
        self.ratio_matrix = ratio_matrix
        self.c_min = c_min
        self.c_max = c_max
        self.step = step
        self.v = v
        self.base_seq_idx = base_phase_seq

        self.num_intersections = len(self.ratio_matrix)
        self.idea_dis_table = []
        self.idea_dis_matrix = []
        self.bias_matrix = []

    def getInterval(self, c):
        return 0.5 * self.v * c

    def getDeltaT(self, idx, seq_idx, c):
        delta_t = 0.5*self.ratio_matrix[idx][0]*c + 0.5*self.ratio_matrix[idx][1]*c
        t_i = (1 - np.sum(self.ratio_matrix[idx])) / len(self.ratio_matrix[idx])

        if seq_idx == 0:
            delta_t += t_i
        elif seq_idx == 1:
            delta_t = -1 * delta_t - t_i
        elif seq_idx == 2:
            delta_t += self.ratio_matrix[idx][2]*c + 2*t_i
        elif seq_idx == 3:
            delta_t += self.ratio_matrix[idx][3]*c + 2*t_i

        return delta_t

    def getIdealDis(self, a_idx, b_idx, a_seq_idx, b_seq_idx, c, n=0):
        # FIXME: error in caclulating delta_t_a
        delta_t_a = self.getDeltaT(a_idx, a_seq_idx, c)
        delta_t_b = self.getDeltaT(b_idx, b_seq_idx, c)

        res = 0.5*v*delta_t_a - 0.5*v*delta_t_b
        while res < 0:
            res += self.getInterval(c)

        return res

    def getIdealDisTable(self):
        '''
        Calculate ideal distance from adjacent base intersection to each non-base intersection with each phase sequence
        '''

        idea_dis_cols = ['cycle']
        for i in range(1, self.num_intersections):
            for j in range(4):
                name = str(i) + '-' + str(j)
                idea_dis_cols.append(name)
        idea_dis_cols.append('interval')

        idea_dis_matrix = []
        idea_dis_table = []

        for t in range(self.c_min, self.c_max+self.step, self.step):
            matrix_row = []
            table_row = [t]

            for i in range(1, self.num_intersections):
                for j in range(4):
                    ideal_dis = self.getIdealDis(0, i, self.base_seq_idx, j, t)
                    matrix_row.append(ideal_dis)

            idea_dis_matrix.append(matrix_row)
            table_row += matrix_row
            table_row.append(self.getInterval(t))
            idea_dis_table.append(table_row)

        idea_dis_df = pd.DataFrame(idea_dis_table, columns=idea_dis_cols)
        self.idea_dis_table = idea_dis_table
        self.idea_dis_matrix = idea_dis_matrix
        return idea_dis_df

    def getBiasTable(self):
        '''
        Calculate bias distance from each real intersection to the adjacent base intersection on the left
        '''

        bias_cols = ['cycle']
        bias_cols += [i for i in range(1, self.num_intersections)]
        bias_cols.append('interval')

        bias_matrix = []
        bias_table = []

        dis = [0]
        for i in range(1, self.num_intersections):
            dis.append(dis[i-1]+self.spacing[i-1])

        for t in range(self.c_min, self.c_max+self.step, self.step):
            matrix_row = []
            table_row = [t]
            interval = self.getInterval(t)

            for i in range(1, self.num_intersections):
                bias = dis[i] % interval

                matrix_row.append(bias)

            table_row += matrix_row
            table_row.append(interval)
            bias_matrix.append(matrix_row)
            bias_table.append(table_row)

        bias_df = pd.DataFrame(bias_table, columns=bias_cols)
        self.bias_matrix = bias_matrix
        return bias_df

    def getOptimalTable(self):
        '''
        Find optimal cycle and sequence with minimun offset ratio
        '''

        self.getIdealDisTable()
        self.getBiasTable()

        offset_ratio_cols = ['cycle']
        for i in range(1, self.num_intersections):
            for j in range(4):
                name = str(i) + '-' + str(j)
                offset_ratio_cols.append(name)

            offset_ratio_cols.append('optimal_seq')

        optimal_table_cols = ['cycle']
        optimal_table_cols += [str(i) for i in range(1, self.num_intersections)]
        optimal_table_cols.append('optimal phase sequence')
        optimal_table_cols.append('max offset ratio sum')

        offset_ratio_matrix = []
        offset_ratio_table = []
        optimal_ratio_matrix = []
        optimal_table = []

        for t in range(len(self.idea_dis_matrix)):

            matrix_row = []
            table_row = [self.idea_dis_table[t][0]]
            optimal_matrix_row = []
            optimal_table_row = [self.idea_dis_table[t][0]]
            optimal_seq = []
            interval = self.idea_dis_table[t][-1]

            for i in range(1, self.num_intersections):
                min_offset_ratio = float('inf')
                optimal_phase_seq_idx = -1

                for j in range(4):

                    diff = self.bias_matrix[t][i-1] - self.idea_dis_matrix[t][(i-1)*4+j]
                    #print("{} - {} = {}".format(bias_matrix[t][i-1], idea_dis_matrix[t][(i-1)*4+j], diff))
                    while diff < 0 and abs((diff+interval)) < abs(diff):
                        diff += interval

                    offset_ratio = 0.5 * diff / interval

                    matrix_row.append(offset_ratio)

                    if np.abs(min_offset_ratio) > np.abs(offset_ratio):
                        min_offset_ratio = offset_ratio
                        optimal_phase_seq_idx = j

                matrix_row.append(optimal_phase_seq_idx)
                optimal_seq.append(optimal_phase_seq_idx)
                optimal_matrix_row.append(min_offset_ratio)

            table_row += matrix_row
            optimal_table_row += optimal_matrix_row
            optimal_table_row.append(optimal_seq)

            # max offset ratio sum
            sum = max(optimal_matrix_row) - min(optimal_matrix_row)
            optimal_table_row.append(sum)

            offset_ratio_table.append(table_row)
            offset_ratio_matrix.append(matrix_row)
            optimal_ratio_matrix.append(optimal_matrix_row)
            optimal_table.append(optimal_table_row)

        optimal_df = pd.DataFrame(optimal_table, columns=optimal_table_cols)
        optimal_data = optimal_df[optimal_df['max offset ratio sum'] == optimal_df['max offset ratio sum'].min()]
        optimal_data = optimal_data.iloc[0]

        return optimal_data
        



# testcase
test = LineCoordinationBase1Phase(spacing, ratio_matrix, c_min, c_max, step, v, 1)
df = test.getIdealDisTable()
df = test.getBiasTable()
df = test.getOptimalTable()
df

cycle                              110
1                            -0.035455
2                                0.115
3                             0.001818
4                             0.139091
optimal phase sequence    [1, 0, 0, 0]
max offset ratio sum          0.174545
Name: 10, dtype: object

对整个计算过程全封装

In [323]:
class LineCoordination:
    def __init__(self, spacing, ratio_matrix, c_min, c_max, step, v):
        '''
        spacing: spacing of intersections in one line
        ratio_matrix: ratios of each intersections, sequence: SNEW
        c_min: minimum cycle, unit: s
        c_max: maximum cycle, unit: s
        step: cycle range will grow by step
        v: green wave speed, unit: m/s

        Valid phase sequence: 
        0: SNEW
        1: NSEW
        2: SENW
        3: SWNE
        '''
        

        self.spacing = spacing
        self.ratio_matrix = ratio_matrix
        self.c_min = c_min
        self.c_max = c_max
        self.step = step
        self.v = v

        self.optimal_all_data = pd.Series()
        self.ideal_pos_list = []

        self.num_intersections = len(self.ratio_matrix)

        self.dis = [0]
        for i in range(1, self.num_intersections):
            self.dis.append(self.dis[i-1] + self.spacing[i-1])

    def getInterval(self, c):
        return 0.5 * self.v * c

    def getDeltaT(self, idx, seq_idx, c):
        delta_t = 0.5*self.ratio_matrix[idx][0]*c + 0.5*self.ratio_matrix[idx][1]*c
        t_i = (1 - np.sum(self.ratio_matrix[idx])) / len(self.ratio_matrix[idx])

        if seq_idx == 0:
            delta_t += t_i
        elif seq_idx == 1:
            delta_t = -1 * delta_t - t_i
        elif seq_idx == 2:
            delta_t += self.ratio_matrix[idx][2]*c + 2*t_i
        elif seq_idx == 3:
            delta_t += self.ratio_matrix[idx][3]*c + 2*t_i

        return delta_t

    def getIdealDis(self, a_idx, b_idx, a_seq_idx, b_seq_idx, c, n=0):
        # FIXME: error in caclulating delta_t_a
        delta_t_a = self.getDeltaT(a_idx, a_seq_idx, c)
        delta_t_b = self.getDeltaT(b_idx, b_seq_idx, c)

        res = 0.5*v*delta_t_a - 0.5*v*delta_t_b
        while res < 0:
            res += self.getInterval(c)

        return res

    def findOptimal(self):
        optimal_all_cols = ['base phase sequence', 'cycle']
        optimal_all_cols += [str(i) for i in range(1, self.num_intersections)]
        optimal_all_cols += ["phase sequence order", 'max offset ratio']

        optimal_all_table = []

        for i in range(4):
            line_phase = LineCoordinationBase1Phase(self.spacing, 
                                                    self.ratio_matrix, 
                                                    self.c_min, 
                                                    self.c_max, 
                                                    self.step, 
                                                    self.v, i)
            
            phase_list = line_phase.getOptimalTable().to_list()
            phase_list.insert(0, i)
            optimal_all_table.append(phase_list)

        optimal_all_df = pd.DataFrame(optimal_all_table, columns=optimal_all_cols)
        optimal_all_df = optimal_all_df[optimal_all_df['max offset ratio'] == optimal_all_df['max offset ratio'].min()]
        optimal_all_data = optimal_all_df.iloc[0]
        self.optimal_all_data = optimal_all_data
        return optimal_all_data

    def getPhaseDiff(self):
        '''
        direction: S to N
        '''
        self.findOptimal()

        phase_seq = self.optimal_all_data['phase sequence order']
        phase_diff_list = []
        dis_diff_list = []
        ratio_s = []
        ideal_pos_list = [0]
        base_phase = 0
        c = self.optimal_all_data['cycle']
        interval = getInterval(c)

        for i in self.ratio_matrix:
            ratio_s.append(i[0])

        for i in range(1, self.num_intersections):
            ideal_dis = getIdealDis(0, i, base_phase, phase_seq[i-1], c)
            ideal_pos = ideal_dis

            # ideal pos
            dis_diff = self.dis[i] - ideal_dis
            while np.abs(dis_diff) > np.abs(dis_diff - interval):
                dis_diff -= interval
                ideal_pos += interval

            ideal_pos_list.append(ideal_pos)
            dis_diff_list.append(dis_diff)

        for i in range(self.num_intersections):
            phase_diff = ideal_pos_list[i]/v - 0.5*c*ratio_s[i]
            while phase_diff > c:
                phase_diff -= c
            while phase_diff < 0:
                phase_diff += c

            phase_diff_list.append(round(phase_diff))

        tmp = [ideal_pos_list, ratio_s, phase_diff_list]
        cols = [i for i in range(self.num_intersections)]
        idx = ['position', 'ratio', 'phase diff']
        df = pd.DataFrame(tmp, columns=cols, index=idx)

        #print(dis_diff_list)
        self.ideal_pos_list = ideal_pos_list

        return df

    def getBandWidth(self, direction=0):
        '''
        direction:
            0: S-N (default)
            1: N-S
        '''
        self.getPhaseDiff()

        c = self.optimal_all_data['cycle']
        interval = self.getInterval(c)

        ratio = []
        for i in self.ratio_matrix:
            if direction == 0:
                ratio.append(i[0])
            elif direction == 1:
                ratio.append(i[1])

        dis_diff = []
        for i in range(self.num_intersections):
            if direction == 0:
                dis_diff.append(self.dis[i] - self.ideal_pos_list[i])
            elif direction == 1:
                dis_diff.append((self.dis[i] - self.ideal_pos_list[i])*-1)

        pos = []
        for i in dis_diff:
            if i > 0:
                pos.append('down')
            elif i == 0:
                pos.append('mid')
            else:
                pos.append('up')

        loss = [np.abs(i / interval) for i in dis_diff]
        offset_ratio = [i / 2 for i in loss]

        up_ratio = []
        down_ratio = []
        for i in range(len(dis_diff)):
            if dis_diff[i] > 0:
                up_ratio.append(0.5*ratio[i] - offset_ratio[i])
                down_ratio.append(0.5*ratio[i] + offset_ratio[i])
            elif dis_diff[i] == 0:
                up_ratio.append(0.5*ratio[i])
                down_ratio.append(0.5*ratio[i])
            else:
                up_ratio.append(0.5*ratio[i] + offset_ratio[i])
                down_ratio.append(0.5*ratio[i] - offset_ratio[i])

            width = min(up_ratio) + min(down_ratio)

        '''
        print(dis_diff)
        print(offset_ratio)
        print(up_ratio)
        print(down_ratio)
        print(width)
        '''

        cols = [str(i) for i in range(self.num_intersections)]
        idx = ['posotion', 'offset ratio', 'ratio on the centerline', 'ratio down the centerline']
        tmp = [pos, offset_ratio, up_ratio, down_ratio]
        df = pd.DataFrame(tmp, columns=cols, index=idx)

        return width, df



# testcase
test_ex = LineCoordination(spacing, ratio_matrix, c_min, c_max, step, v)
df = test_ex.getPhaseDiff()
#width, df = test_ex.getBandWidth(0)
#print("width: {}".format(width))
df

C:\Users\gtl11\AppData\Local\Temp/ipykernel_25556/3251903912.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.optimal_all_data = pd.Series()


,0,1,2,3,4
position,0.00,499.8,877.10,1293.6,1450.40
ratio,0.34,0.3,0.32,0.3,0.36
phase diff,81.00,35.0,72.00,17.0,29.00
